In [ ]:

#Install Detectron2 from the source
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
#Set up directory and mounting from GDrive
import os
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#import libraries
import glob
import json
import os
import cv2
import numpy as np
import torch
print(torch.__version__)
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode


## Loading the model

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import build_detection_test_loader, MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg

# Register the dataset only if it's not already registered
#dataset_name = "waterbodies_test"


def get_waterbodies_dicts(img_dir, annotations_json):
    json_file = os.path.join(img_dir, annotations_json)
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns['images']):
        record = {}
        filename = os.path.join(img_dir, v['file_name'])
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width

        annos = [anno for anno in imgs_anns['annotations'] if anno['image_id'] == v['id']]
        objs = []
        for anno in annos:
            obj = {
                "bbox": anno['bbox'],
                "bbox_mode": BoxMode.XYWH_ABS,
                "segmentation": anno['segmentation'],
                "category_id": anno['category_id'],
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

# Paths to the train and test annotation files

# Register the datasets
# Paths to the train and test annotation files
annotation_file_test = '/content/drive/MyDrive/Project/annotations_test.json'
annotation_file_train = '/content/drive/MyDrive/Project/annotations_train.json'

# Register the datasets
DatasetCatalog.register("waterbodies_train", lambda: get_waterbodies_dicts("/content/drive/MyDrive/Project/images/", annotation_file_train))
MetadataCatalog.get("waterbodies_train").set(thing_classes=["waterbody"])

DatasetCatalog.register("waterbodies_test", lambda: get_waterbodies_dicts("/content/drive/MyDrive/Project/images/", annotation_file_test))
MetadataCatalog.get("waterbodies_test").set(thing_classes=["waterbody"])






In [ ]:
# Initialize model
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/Project/model_final.pth"

#threshold for predictions
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.DATASETS.TEST = ("waterbodies_test",)
cfg.DATASETS.TRAIN = ("waterbodies_train",)  # Make sure no default COCO dataset is referenced
cfg.DATASETS.VAL = ("waterbodies_test",)  # Make sure no default COCO dataset is referenced
# cfg.INPUT.MIN_SIZE_TRAIN = (512,)  # Only one size, no choice needed
# cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
# cfg.INPUT.MAX_SIZE_TRAIN = 512  # No need to allow any larger sizes
# cfg.INPUT.MIN_SIZE_TEST = 512  # Same as training
# cfg.INPUT.MAX_SIZE_TEST = 512  # Same as training
# Setting up the test data loader and evaluator
val_loader = build_detection_test_loader(cfg, "waterbodies_test")
evaluator = COCOEvaluator("waterbodies_test", cfg, False, output_dir="/content/drive/MyDrive/Project/output/")


trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
# results = inference_on_dataset(trainer.model, val_loader, evaluator)
# print(results)
predictor = DefaultPredictor(cfg)

In [ ]:
#install Google Earth Engine API
!pip install earthengine-api

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project = 'southern-bonsai-435419-q2')

## Set the Parameters for the outputs

In [ ]:

## Set Area and buffer zone
#Mareb
longitude = 45.230503
latitude = 15.376969
bufferzone = 2500
descrip = 'Mareb_dam_landsat'
#Mareb
# longitude = 45.230503
# latitude = 15.376969
# bufferzone = 3000
# descrip = 'Mareb_dam_sentinel'

#Set parameters for satellite images
# Date range if landsat
start_date = '1984-01-01'
end_date = '2022-12-31'
date_range = '1984-2022'



#Wadi Wajar Abyan
# longitude = 46.156746
# latitude = 13.941848
# bufferzone = 2500
# descrip = 'Wadi_wajar_landsat'

#Bani Matar Sana'a
# longitude = 43.994586
# latitude = 15.203803
# bufferzone = 800
# descrip = 'Bani_matar_landsat'

#Wadi Wajar Abyan
# longitude = 46.156746
# latitude = 13.941848
# bufferzone = 1000
#descrip = 'Wadi_wajar_sentinel'

#if sentinel
# start_date = '2016-01-01'  # Sentinel-2 operation start
# end_date = '2022-12-31'
# date_range = '2016-2022'
# descrip = 'Mareb_dam_sentinel'


## Landsat Images

In [ ]:
import ee
import time
#adapted from https://github.com/csaybar/EEwPython/blob/master/10_Export.ipynb
# Initialize the Earth Engine API
ee.Initialize()



point = ee.Geometry.Point([longitude, latitude])
buffered_point = point.buffer(bufferzone)
region = buffered_point.bounds()

# Load and merge the updated image collections using Collection 2 data
landsat5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_TOA').filterDate('1984-01-01', '2012-05-05')
landsat7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA').filterDate('1999-01-01', '2022-12-31')
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2013-01-01', '2022-12-31')
full_collection = landsat5.merge(landsat7).merge(landsat8)

# Filter out images with more than 10% cloud cover and within the region
filtered_collection = full_collection.filter(ee.Filter.lt('CLOUD_COVER', 10)).filterBounds(region).select(['B4', 'B3', 'B2'])

# Generate a list of months between the start and end dates
start = ee.Date(start_date)
end = ee.Date(end_date)
n_months = end.difference(start, 'month').round()
months = ee.List.sequence(0, n_months.subtract(1)).map(lambda n: start.advance(n, 'month'))

# Function to get one image per month
def get_image_for_month(month):
    month = ee.Date(month)
    month_end = month.advance(1, 'month')
    # Filter images in the month
    images_in_month = filtered_collection.filterDate(month, month_end)
    # Get the first image
    image = ee.Image(images_in_month.first())
    # Check if image exists
    return ee.Algorithms.If(image, image.set('system:time_start', month.millis()), None)

# Map the function over the list of months to create a list of images
images_list = months.map(get_image_for_month)

# Create an ImageCollection from the list of images and filter out None values
monthly_images = ee.ImageCollection.fromImages(images_list).filter(ee.Filter.notNull(['system:time_start']))

# Function to scale and cast images
def scale_and_cast(image):
    return image.multiply(255).byte()  # Scale and convert to uint8

# Apply the function to each image in the collection
monthly_images = monthly_images.map(scale_and_cast)



# Create and start the video export task
task = ee.batch.Export.video.toDrive(
    collection=monthly_images,
    description=descrip,
    dimensions=1080,
    framesPerSecond=12,
    region=region.coordinates().getInfo(),
    folder='videosIn2',
    maxPixels=1e13,
    maxFrames=10000  # Increase the maxFrames as needed
)
task.start()

# Monitor the task
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)  # Sleep for 10 seconds before polling again

print("Video export complete.")


## Sentinel-2 images

In [ ]:
import ee
import time

# Initialize the Earth Engine API
ee.Initialize()

# Define the geographic location and buffer
# longitude = 46.156746
# latitude = 13.941848

# video_name = 'Abyan3_Sentinel2'

point = ee.Geometry.Point([longitude, latitude])
buffered_point = point.buffer(bufferzone).bounds()  # Define a 2km buffer around the point
region = buffered_point.bounds()

# Date range
# start_date = '2015-06-23'  # Sentinel-2 operation start
# end_date = '2022-12-31'

# Load Sentinel-2 ImageCollection
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(region) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Filter images based on cloud cover

# Select the RGB bands and scale
def prepare_image(image):
    return image.visualize(bands=['B4', 'B3', 'B2'], max=3000).clip(region)

# Apply the function to each image in the collection
prepared_collection = collection.map(prepare_image)

# Create and start the video export task
task = ee.batch.Export.video.toDrive(**{
    'collection': prepared_collection,
    'description': descrip,
    'dimensions': 1080,
    'framesPerSecond': 12,
    'region': region.coordinates().getInfo(),
    'folder': 'VideosIn2',
    'maxFrames': 10000
})
task.start()

# Monitor the task
while task.active():
    print('Polling for task (id: {})'.format(task.id))
    time.sleep(10)  # Sleep for 10 seconds before polling again

print("Video export complete.")

## Importing exported video from GDrive and deploying predictor

In [ ]:
# Adapted from https://stackoverflow.com/questions/60663073/how-can-i-properly-run-detectron2-on-videos

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import necessary libraries
import numpy as np
import tqdm
import cv2
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog


# Extract video properties
descrip = descrip  # Set video description/name
video_path = '/content/drive/MyDrive/VideosIn2/' + descrip + '.mp4'
video = cv2.VideoCapture(video_path)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
output_video_path = '/content/drive/MyDrive/VideosOut2/' + descrip + '_out.mp4'
video_writer = cv2.VideoWriter(
    output_video_path,
    fourcc=cv2.VideoWriter_fourcc(*"mp4v"),
    fps=float(frames_per_second),
    frameSize=(width, height),
    isColor=True,
)

# Initialize visualizer
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
v = VideoVisualizer(metadata, ColorMode.IMAGE)

# Define start date and generate list of dates for each frame
# Define start and end years
start_year = 2016
end_year = 2022
num_years = end_year - start_year + 1  # Inclusive of both start and end years
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frames_per_year = num_frames / num_years

def runOnVideo(video, maxFrames):
    """Runs the predictor on every frame in the video (up to maxFrames),
    and yields the frame index, frame, visualization, predicted mask, and predicted water pixel count.
    """
    readFrames = 0
    while True:
        hasFrame, frame = video.read()
        if not hasFrame:
            break

        # Get prediction results for this frame
        outputs = predictor(frame)

        # Get predicted masks
        pred_masks = outputs['instances'].pred_masks.cpu().numpy()
        if pred_masks.size == 0:
            predicted_mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
        else:
            predicted_mask = (np.sum(pred_masks, axis=0) >= 1).astype(np.uint8) * 255

        # Count the number of pixels in the predicted mask
        predicted_water_pixels = np.sum(predicted_mask == 255)

        # Draw a visualization of the predictions using the video visualizer
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Convert Matplotlib RGB format to OpenCV BGR format
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield readFrames, frame, visualization, predicted_mask, predicted_water_pixels

        readFrames += 1
        if readFrames >= maxFrames:
            break

# Initialize a list to store results
results = []

# Enumerate the frames of the video
for frame_idx, frame, visualization, predicted_mask, predicted_water_pixels in tqdm.tqdm(
    runOnVideo(video, num_frames), total=num_frames
):
    # Map frame indices to years
    year = start_year + int(frame_idx * num_years / num_frames)
    if year > end_year:
        year = end_year  # Ensure the year does not exceed the end year

    # Save images for specific years
    if year in [2016, 2017, 2018, 2019, 2020,2021, 2022]:
        # Save original frame
        cv2.imwrite(f'/content/drive/MyDrive/VideosOut2/Images/{descrip}_original_{year}.png', frame)
        # Save annotated frame
        cv2.imwrite(f'/content/drive/MyDrive/VideosOut2/Images/{descrip}_annotated_{year}.png', visualization)
        # Save mask frame
        cv2.imwrite(f'/content/drive/MyDrive/VideosOut2/Images/{descrip}_mask_{year}.png', predicted_mask)

    # Write to video file
    video_writer.write(visualization)

    # Collect results
    results.append({'year': year, 'predicted_water_pixels': predicted_water_pixels})

# Release resources
video.release()
video_writer.release()
cv2.destroyAllWindows()

# Create a dataframe from the results and save to CSV
df = pd.DataFrame(results)
csv_output_path = '/content/drive/MyDrive/VideosOut2/Data/predicted_water_pixel_counts_' + descrip + '.csv'
df.to_csv(csv_output_path, index=False)
print(df)


## visualise the change over time in terms of predicted pixels

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv('/content/drive/MyDrive/VideosOut2/Data/predicted_water_pixel_counts_' + descrip + '.csv')

# Group the data by year and calculate the average
df_grouped = df.groupby('year')['predicted_water_pixels'].mean().reset_index()


# Plotting the data
plt.figure(figsize=(12, 6))
plt.plot(df_grouped['year'], df_grouped['predicted_water_pixels'])
plt.title('Predicted Water Pixels Over Years (' + descrip + ')')
plt.xlabel('Year')
plt.ylabel('Average Predicted Water Pixels')
plt.grid(True)
plt.tight_layout()

# Save the plot as a JPEG
plt.savefig('/content/drive/MyDrive/VideosOut2/Plots/predicted_water_pixel_counts_' + descrip + '_aggregated.jpg', format='jpg')
plt.show()